In [1]:
import sqlite3, os, time
import pandas as pd
import numpy as np

In [2]:
path = os.getcwd()+'/'

## preprocess data

In [3]:
data = pd.read_csv(path+'card_transactions.csv')
data.rename(columns={"Merch Description": "Merch_Description", 
                      "Merchant State": "Merchant_State",
                      'Merchant Zip': 'Merchant_Zip'},
           inplace = True)
dates = data['Date'].unique()
data['date_index'] = data['Date'].apply(lambda x: int(np.where(dates==x)[0]))

In [4]:
data = data.query('Transtype == "P"')

In [5]:
data.head()

,Recordnum,Cardnum,Date,Merchantnum,Merch_Description,Merchant_State,Merchant_Zip,Transtype,Amount,Fraud,date_index
0,1,5142190439,2010-01-01,5509006296254,FEDEX SHP 12/23/09 AB#,TN,38118.0,P,3.62,0,0
1,2,5142183973,2010-01-01,61003026333,SERVICE MERCHANDISE #81,MA,1803.0,P,31.42,0,0
2,3,5142131721,2010-01-01,4503082993600,OFFICE DEPOT #191,MD,20706.0,P,178.49,0,0
3,4,5142148452,2010-01-01,5509006296254,FEDEX SHP 12/28/09 AB#,TN,38118.0,P,3.62,0,0
4,5,5142190439,2010-01-01,5509006296254,FEDEX SHP 12/23/09 AB#,TN,38118.0,P,3.62,0,0


### exchanging peso into dollar

In [13]:
data['Amount'].max()

3102045.53

In [15]:
data.query('Amount == 3102045.53')

,Recordnum,Cardnum,Date,Merchantnum,Merch_Description,Merchant_State,Merchant_Zip,Transtype,Amount,Fraud,date_index
52593,52594,5142189135,2010-07-13,NaN,INTERMEXICO,NaN,NaN,P,3102045.53,0,193


In [18]:
data.loc[52593,'Amount'] = data.loc[52593,'Amount']*0.079

In [19]:
data.loc[52593,:]

Recordnum                  52594
Cardnum               5142189135
Date                  2010-07-13
Merchantnum                  NaN
Merch_Description    INTERMEXICO
Merchant_State               NaN
Merchant_Zip                 NaN
Transtype                      P
Amount                    245062
Fraud                          0
date_index                   193
Name: 52593, dtype: object

In [22]:
data['Amount'].max()

245061.59686999998

### merchnum first occurence 

must be done before inserting missing values

In [23]:
global_list = []
def in_list(x):
    global global_list
    if x not in global_list:
        global_list.append(x)
        return False
    else:
        return True   

In [24]:
data['new_merch'] = data['Merchantnum'].apply(lambda x: 0 if in_list(x) == True else 1)

### heuristic time series difference

#### Monday=0, Sunday=6
#### January=1, December=12

In [25]:
data['weekday'] = pd.to_datetime(data['Date']).dt.dayofweek
data['month'] = pd.to_datetime(data['Date']).dt.month

In [26]:
avg_weekday = {}
avg_month = {}
for i in range(7):
    avg_weekday[i]=np.mean(data.query('weekday == @i')['Amount'])
for i in range(1,13):
    avg_month[i]=np.mean(data.query('month == @i')['Amount'])

In [27]:
avg_weekday

{0: 384.8559583198359,
 1: 420.1150861923684,
 2: 439.1792176434506,
 3: 358.7174506086609,
 4: 368.10546369376476,
 5: 379.418336781949,
 6: 387.3252082501663}

In [28]:
avg_month

{1: 353.5521079009434,
 2: 355.14537765131917,
 3: 352.47009212640603,
 4: 358.138674025974,
 5: 368.41028992021575,
 6: 391.16843580273735,
 7: 423.8506113237427,
 8: 426.6661320240043,
 9: 456.6117632628028,
 10: 479.5257552392249,
 11: 431.28618665756693,
 12: 400.37695762081785}

In [29]:
data.head()

,Recordnum,Cardnum,Date,Merchantnum,Merch_Description,Merchant_State,Merchant_Zip,Transtype,Amount,Fraud,date_index,new_merch,weekday,month
0,1,5142190439,2010-01-01,5509006296254,FEDEX SHP 12/23/09 AB#,TN,38118.0,P,3.62,0,0,1,4,1
1,2,5142183973,2010-01-01,61003026333,SERVICE MERCHANDISE #81,MA,1803.0,P,31.42,0,0,1,4,1
2,3,5142131721,2010-01-01,4503082993600,OFFICE DEPOT #191,MD,20706.0,P,178.49,0,0,1,4,1
3,4,5142148452,2010-01-01,5509006296254,FEDEX SHP 12/28/09 AB#,TN,38118.0,P,3.62,0,0,0,4,1
4,5,5142190439,2010-01-01,5509006296254,FEDEX SHP 12/23/09 AB#,TN,38118.0,P,3.62,0,0,0,4,1


In [30]:
week_diff = []
month_diff = []
for i,row in data.iterrows():
    week_diff.append(row['Amount']-avg_weekday[row['weekday']])
    month_diff.append(row['Amount']-avg_month[row['month']])

In [31]:
data['Amount_week_diff'] = week_diff
data['Amount_month_diff'] = month_diff

### missing value processing

In [32]:
summary_table = data.describe(include='all').transpose().iloc[:,:2]
summary_table['populated_rate'] = summary_table['count']/len(data)
summary_table

,count,unique,populated_rate
Recordnum,96353,NaN,1
Cardnum,96353,NaN,1
Date,96353,365,1
Merchantnum,93154,13089,0.966799
Merch_Description,96353,12966,1
Merchant_State,95332,59,0.989404
Merchant_Zip,92052,NaN,0.955362
Transtype,96353,1,1
Amount,96353,NaN,1
Fraud,96353,NaN,1


Since we assign each na different value to prevent inflating the burst in `Merchantnum`, we need isna identifier to prevent inflating different `Merchant_State` and `Merchant_Zip`

In [33]:
data['Merchantnum_isna'] = data['Merchantnum'].isna().astype('int')
data['Merchant_State_isna'] = data['Merchant_State'].isna().astype('int')
data['Merchant_Zip_isna'] = data['Merchant_Zip'].isna().astype('int')

In [34]:
1-np.mean(data['Merchant_Zip_isna'])

0.95536205411352

In [35]:
data['for_na'] = np.array(['n_'+str(d) for d in data['Recordnum']])

In [36]:
data['Merchantnum'].fillna(data['for_na'],inplace=True)
data['Merchant_State'].fillna(data['for_na'],inplace=True)
data['Merchant_Zip'].fillna(data['for_na'],inplace=True)

In [37]:
data.query('Merchant_Zip.isna()')

,Recordnum,Cardnum,Date,Merchantnum,Merch_Description,Merchant_State,Merchant_Zip,Transtype,Amount,Fraud,date_index,new_merch,weekday,month,Amount_week_diff,Amount_month_diff,Merchantnum_isna,Merchant_State_isna,Merchant_Zip_isna,for_na


In [38]:
data.tail(10)

,Recordnum,Cardnum,Date,Merchantnum,Merch_Description,Merchant_State,Merchant_Zip,Transtype,Amount,Fraud,date_index,new_merch,weekday,month,Amount_week_diff,Amount_month_diff,Merchantnum_isna,Merchant_State_isna,Merchant_Zip_isna,for_na
96698,96699,5142187448,2010-12-31,5725000466504,CDW*GOVERNMENT INC,IL,60061,P,2231.08,0,364,0,4,12,1862.974536,1830.703042,0,0,0,n_96699
96699,96700,5142160778,2010-12-31,08-0616075333,COMPUTER STORE OF CORVLS,OR,97330,P,243.15,1,364,0,4,12,-124.955464,-157.226958,0,0,0,n_96700
96700,96701,5142226979,2010-12-31,602608969534,FISHER SCI ATL,GA,30091,P,32.04,0,364,0,4,12,-336.065464,-368.336958,0,0,0,n_96701
96701,96702,5142196337,2010-12-31,467619770330,STAY ONLINE,NC,27560,P,147.35,0,364,0,4,12,-220.755464,-253.026958,0,0,0,n_96702
96702,96703,5142221253,2010-12-31,5725000466504,CDW*GOVERNMENT INC,IL,60061,P,425.00,0,364,0,4,12,56.894536,24.623042,0,0,0,n_96703
96703,96704,5142276053,2010-12-31,3500000006160,BEST BUY 00001610,KY,41042,P,84.79,0,364,1,4,12,-283.315464,-315.586958,0,0,0,n_96704
96704,96705,5142225701,2010-12-31,8090710030950,MARKUS OFFICE SUPPLIES,OH,45248,P,118.75,0,364,0,4,12,-249.355464,-281.626958,0,0,0,n_96705
96705,96706,5142226486,2010-12-31,4503057341100,"TECH PAC, INC",OH,45150,P,363.56,0,364,0,4,12,-4.545464,-36.816958,0,0,0,n_96706
96706,96707,5142244619,2010-12-31,8834000695412,BUY.COM,CA,92656,P,2202.03,0,364,0,4,12,1833.924536,1801.653042,0,0,0,n_96707
96707,96708,5142243247,2010-12-31,9108347680006,STAPLES NATIONAL #471,NJ,7606,P,554.64,0,364,0,4,12,186.534536,154.263042,0,0,0,n_96708


## Load data into sqlite and create variables

In [39]:
conn = sqlite3.connect(path+'fraud.sqlite')
#drop tables
conn.cursor().execute('drop table if exists card_transactions;')
data.to_sql(name='card_transactions', con = conn, index = False, chunksize = None)

In [40]:
def pysql(sqlcode):
    return pd.read_sql_query(sqlcode, conn)

In [41]:
sql_code = '''
select * from card_transactions limit 0,3;
'''

In [42]:
pysql(sql_code)

,Recordnum,Cardnum,Date,Merchantnum,Merch_Description,Merchant_State,Merchant_Zip,Transtype,Amount,Fraud,date_index,new_merch,weekday,month,Amount_week_diff,Amount_month_diff,Merchantnum_isna,Merchant_State_isna,Merchant_Zip_isna,for_na
0,1,5142190439,2010-01-01,5509006296254,FEDEX SHP 12/23/09 AB#,TN,38118.0,P,3.62,0,0,1,4,1,-364.485464,-349.932108,0,0,0,n_1
1,2,5142183973,2010-01-01,61003026333,SERVICE MERCHANDISE #81,MA,1803.0,P,31.42,0,0,1,4,1,-336.685464,-322.132108,0,0,0,n_2
2,3,5142131721,2010-01-01,4503082993600,OFFICE DEPOT #191,MD,20706.0,P,178.49,0,0,1,4,1,-189.615464,-175.062108,0,0,0,n_3


In [43]:
pysql('select count(distinct date_index) as num_unique_dates from card_transactions;')

,num_unique_dates
0,365


In [44]:
pysql('select count(*) as length from card_transactions;')

,length
0,96353


In [45]:
sql_code = '''
SELECT t1.Recordnum,

--- 0. basic statistics
-- 0.0. avg amount 30, 15, 7, 3, 1

AVG(t2.Amount) As cardnum_amount_avg_30,
AVG(CASE WHEN t1.date_index - t2.date_index <= 14 THEN t2.Amount ELSE NULL END) As cardnum_amount_avg_15,
AVG(CASE WHEN t1.date_index - t2.date_index <= 6 THEN t2.Amount ELSE NULL END) As cardnum_amount_avg_7,
AVG(CASE WHEN t1.date_index - t2.date_index <= 2 THEN t2.Amount ELSE NULL END) As cardnum_amount_avg_3,
AVG(CASE WHEN t1.date_index - t2.date_index <= 0 THEN t2.Amount ELSE NULL END) As cardnum_amount_avg_1,

-- 0.1. max amount 30, 15, 7, 3, 1

MAX(t2.Amount) As cardnum_amount_max_30,
MAX(CASE WHEN t1.date_index - t2.date_index <= 14 THEN t2.Amount ELSE NULL END) As cardnum_amount_max_15,
MAX(CASE WHEN t1.date_index - t2.date_index <= 6 THEN t2.Amount ELSE NULL END) As cardnum_amount_max_7,
MAX(CASE WHEN t1.date_index - t2.date_index <= 2 THEN t2.Amount ELSE NULL END) As cardnum_amount_max_3,
MAX(CASE WHEN t1.date_index - t2.date_index <= 0 THEN t2.Amount ELSE NULL END) As cardnum_amount_max_1,

-- 0.2. sum amount 30, 15, 7, 3, 1

SUM(t2.Amount) As cardnum_amount_sum_30,
SUM(CASE WHEN t1.date_index - t2.date_index <= 14 THEN t2.Amount ELSE NULL END) As cardnum_amount_sum_15,
SUM(CASE WHEN t1.date_index - t2.date_index <= 6 THEN t2.Amount ELSE NULL END) As cardnum_amount_sum_7,
SUM(CASE WHEN t1.date_index - t2.date_index <= 2 THEN t2.Amount ELSE NULL END) As cardnum_amount_sum_3,
SUM(CASE WHEN t1.date_index - t2.date_index <= 0 THEN t2.Amount ELSE NULL END) As cardnum_amount_sum_1,

-- 0.3. avg amount week_diff 30, 15, 7, 3, 1

AVG(t2.Amount_week_diff) As cardnum_amount_week_diff_avg_30,
AVG(CASE WHEN t1.date_index - t2.date_index <= 14 THEN t2.Amount_week_diff ELSE NULL END) 
As cardnum_amount_week_diff_avg_15,
AVG(CASE WHEN t1.date_index - t2.date_index <= 6 THEN t2.Amount_week_diff ELSE NULL END) 
As cardnum_amount_week_diff_avg_7,
AVG(CASE WHEN t1.date_index - t2.date_index <= 2 THEN t2.Amount_week_diff ELSE NULL END) 
As cardnum_amount_week_diff_avg_3,
AVG(CASE WHEN t1.date_index - t2.date_index <= 0 THEN t2.Amount_week_diff ELSE NULL END) 
As cardnum_amount_week_diff_avg_1,

-- 0.4. max amount week_diff 30, 15, 7, 3, 1

MAX(t2.Amount_week_diff) As cardnum_amount_week_diff_max_30,
MAX(CASE WHEN t1.date_index - t2.date_index <= 14 THEN t2.Amount_week_diff ELSE NULL END) 
As cardnum_amount_week_diff_max_15,
MAX(CASE WHEN t1.date_index - t2.date_index <= 6 THEN t2.Amount_week_diff ELSE NULL END) 
As cardnum_amount_week_diff_max_7,
MAX(CASE WHEN t1.date_index - t2.date_index <= 2 THEN t2.Amount_week_diff ELSE NULL END) 
As cardnum_amount_week_diff_max_3,
MAX(CASE WHEN t1.date_index - t2.date_index <= 0 THEN t2.Amount_week_diff ELSE NULL END) 
As cardnum_amount_week_diff_max_1,

-- 0.5. sum amount week_diff 30, 15, 7, 3, 1

SUM(t2.Amount_week_diff) As cardnum_amount_week_diff_sum_30,
SUM(CASE WHEN t1.date_index - t2.date_index <= 14 THEN t2.Amount_week_diff ELSE NULL END) 
As cardnum_amount_week_diff_sum_15,
SUM(CASE WHEN t1.date_index - t2.date_index <= 6 THEN t2.Amount_week_diff ELSE NULL END) 
As cardnum_amount_week_diff_sum_7,
SUM(CASE WHEN t1.date_index - t2.date_index <= 2 THEN t2.Amount_week_diff ELSE NULL END) 
As cardnum_amount_week_diff_sum_3,
SUM(CASE WHEN t1.date_index - t2.date_index <= 0 THEN t2.Amount_week_diff ELSE NULL END) 
As cardnum_amount_week_diff_sum_1,

-- 0.6. avg amount month_diff 30, 15, 7, 3, 1

AVG(t2.Amount_month_diff) As cardnum_amount_month_diff_avg_30,
AVG(CASE WHEN t1.date_index - t2.date_index <= 14 THEN t2.Amount_month_diff ELSE NULL END) 
As cardnum_amount_month_diff_avg_15,
AVG(CASE WHEN t1.date_index - t2.date_index <= 6 THEN t2.Amount_month_diff ELSE NULL END) 
As cardnum_amount_month_diff_avg_7,
AVG(CASE WHEN t1.date_index - t2.date_index <= 2 THEN t2.Amount_month_diff ELSE NULL END) 
As cardnum_amount_month_diff_avg_3,
AVG(CASE WHEN t1.date_index - t2.date_index <= 0 THEN t2.Amount_month_diff ELSE NULL END) 
As cardnum_amount_month_diff_avg_1,

-- 0.7. max amount month_diff 30, 15, 7, 3, 1

MAX(t2.Amount_month_diff) As cardnum_amount_month_diff_max_30,
MAX(CASE WHEN t1.date_index - t2.date_index <= 14 THEN t2.Amount_month_diff ELSE NULL END) 
As cardnum_amount_month_diff_max_15,
MAX(CASE WHEN t1.date_index - t2.date_index <= 6 THEN t2.Amount_month_diff ELSE NULL END) 
As cardnum_amount_month_diff_max_7,
MAX(CASE WHEN t1.date_index - t2.date_index <= 2 THEN t2.Amount_month_diff ELSE NULL END) 
As cardnum_amount_month_diff_max_3,
MAX(CASE WHEN t1.date_index - t2.date_index <= 0 THEN t2.Amount_month_diff ELSE NULL END) 
As cardnum_amount_month_diff_max_1,

-- 0.8. sum amount month_diff 30, 15, 7, 3, 1

SUM(t2.Amount_month_diff) As cardnum_amount_month_diff_sum_30,
SUM(CASE WHEN t1.date_index - t2.date_index <= 14 THEN t2.Amount_month_diff ELSE NULL END) 
As cardnum_amount_month_diff_sum_15,
SUM(CASE WHEN t1.date_index - t2.date_index <= 6 THEN t2.Amount_month_diff ELSE NULL END) 
As cardnum_amount_month_diff_sum_7,
SUM(CASE WHEN t1.date_index - t2.date_index <= 2 THEN t2.Amount_month_diff ELSE NULL END) 
As cardnum_amount_month_diff_sum_3,
SUM(CASE WHEN t1.date_index - t2.date_index <= 0 THEN t2.Amount_month_diff ELSE NULL END) 
As cardnum_amount_month_diff_sum_1,

--- 1. burst
-- 1.0. counting the same card number in the past 30,15,7,3,1 days

COUNT(*) AS cardnum_30,
SUM(CASE WHEN t1.date_index - t2.date_index <= 14 THEN 1 ELSE NULL END) AS cardnum_15,
SUM(CASE WHEN t1.date_index - t2.date_index <= 6 THEN 1 ELSE NULL END) AS cardnum_7,
SUM(CASE WHEN t1.date_index - t2.date_index <= 2 THEN 1 ELSE NULL END) AS cardnum_3,
SUM(CASE WHEN t1.date_index - t2.date_index <= 0 THEN 1 ELSE NULL END) AS cardnum_1,

--- 2. new merchant
-- 2.0. counting new merchants given card number in the past 30,15,7,3,1 days

SUM(t2.new_merch) AS newmerch_30,
SUM(CASE WHEN t1.date_index - t2.date_index <= 14 THEN t2.new_merch ELSE NULL END) AS newmerch_15,
SUM(CASE WHEN t1.date_index - t2.date_index <= 6 THEN t2.new_merch ELSE NULL END) AS newmerch_7,
SUM(CASE WHEN t1.date_index - t2.date_index <= 2 THEN t2.new_merch ELSE NULL END) AS newmerch_3,
SUM(CASE WHEN t1.date_index - t2.date_index <= 0 THEN t2.new_merch ELSE NULL END) AS newmerch_1,

--- 3. Merchantnum
-- 3.0. how many different Merchantnum for a given Cardnum in 30,15,7,3,1 days 

COUNT(DISTINCT t2.Merchantnum) AS cardnum_merchantnum_30,
COUNT(DISTINCT CASE WHEN t1.date_index - t2.date_index <= 14 THEN t2.Merchantnum ELSE NULL END) 
AS cardnum_merchantnum_15,
COUNT(DISTINCT CASE WHEN t1.date_index - t2.date_index <= 6 THEN t2.Merchantnum ELSE NULL END) 
AS cardnum_merchantnum_7,
COUNT(DISTINCT CASE WHEN t1.date_index - t2.date_index <= 2 THEN t2.Merchantnum ELSE NULL END) 
AS cardnum_merchantnum_3,
COUNT(DISTINCT CASE WHEN t1.date_index - t2.date_index <= 0 THEN t2.Merchantnum ELSE NULL END) 
AS cardnum_merchantnum_1,

-- 3.1. how many originally na Merchantnum for a given Cardnum in 30,15,7,3,1 days 

SUM(t2.Merchantnum_isna) AS cardnum_merchantnum_isna_30,
SUM(CASE WHEN t1.date_index - t2.date_index <= 14 THEN t2.Merchantnum_isna ELSE NULL END) 
AS cardnum_merchantnum_isna_15,
SUM(CASE WHEN t1.date_index - t2.date_index <= 6 THEN t2.Merchantnum_isna ELSE NULL END) 
AS cardnum_merchantnum_isna_7,
SUM(CASE WHEN t1.date_index - t2.date_index <= 2 THEN t2.Merchantnum_isna ELSE NULL END) 
AS cardnum_merchantnum_isna_3,
SUM(CASE WHEN t1.date_index - t2.date_index <= 0 THEN t2.Merchantnum_isna ELSE NULL END) 
AS cardnum_merchantnum_isna_1,

--- 4. zip
-- 4.0. how many different zip for a given Cardnum in 30,15,7,3,1 days 

COUNT(DISTINCT t2.Merchant_Zip) AS cardnum_zip_30,
COUNT(DISTINCT CASE WHEN t1.date_index - t2.date_index <= 14 THEN t2.Merchant_Zip ELSE NULL END) 
AS cardnum_zip_15,
COUNT(DISTINCT CASE WHEN t1.date_index - t2.date_index <= 6 THEN t2.Merchant_Zip ELSE NULL END) 
AS cardnum_zip_7,
COUNT(DISTINCT CASE WHEN t1.date_index - t2.date_index <= 2 THEN t2.Merchant_Zip ELSE NULL END) 
AS cardnum_zip_3,
COUNT(DISTINCT CASE WHEN t1.date_index - t2.date_index <= 0 THEN t2.Merchant_Zip ELSE NULL END) 
AS cardnum_zip_1,

-- 4.1. how many originally na zip for a given Cardnum in 30,15,7,3,1 days 

SUM(t2.Merchant_Zip_isna) AS cardnum_zip_isna_30,
SUM(DISTINCT CASE WHEN t1.date_index - t2.date_index <= 14 THEN t2.Merchant_Zip_isna ELSE NULL END) 
AS cardnum_zip_isna_15,
SUM(DISTINCT CASE WHEN t1.date_index - t2.date_index <= 6 THEN t2.Merchant_Zip_isna ELSE NULL END) 
AS cardnum_zip_isna_7,
SUM(DISTINCT CASE WHEN t1.date_index - t2.date_index <= 2 THEN t2.Merchant_Zip_isna ELSE NULL END) 
AS cardnum_zip_isna_3,
SUM(DISTINCT CASE WHEN t1.date_index - t2.date_index <= 0 THEN t2.Merchant_Zip_isna ELSE NULL END) 
AS cardnum_zip_isna_1,

--- 5. states
-- 5.0. how many different states for a given Cardnum in 30,15,7,3,1 days 

COUNT(DISTINCT t2.Merchant_State) AS cardnum_state_30,
COUNT(DISTINCT CASE WHEN t1.date_index - t2.date_index <= 14 THEN t2.Merchant_State ELSE NULL END) 
AS cardnum_state_15,
COUNT(DISTINCT CASE WHEN t1.date_index - t2.date_index <= 6 THEN t2.Merchant_State ELSE NULL END) 
AS cardnum_state_7,
COUNT(DISTINCT CASE WHEN t1.date_index - t2.date_index <= 2 THEN t2.Merchant_State ELSE NULL END) 
AS cardnum_state_3,
COUNT(DISTINCT CASE WHEN t1.date_index - t2.date_index <= 0 THEN t2.Merchant_State ELSE NULL END) 
AS cardnum_state_1,

-- 5.1. how many originally na states for a given Cardnum in 30,15,7,3,1 days 

SUM(t2.Merchant_State_isna) AS cardnum_state_isna_30,
SUM(CASE WHEN t1.date_index - t2.date_index <= 14 THEN t2.Merchant_State_isna ELSE NULL END) 
AS cardnum_state_isna_15,
SUM(CASE WHEN t1.date_index - t2.date_index <= 6 THEN t2.Merchant_State_isna ELSE NULL END) 
AS cardnum_state_isna_7,
SUM(CASE WHEN t1.date_index - t2.date_index <= 2 THEN t2.Merchant_State_isna ELSE NULL END) 
AS cardnum_state_isna_3,
SUM(CASE WHEN t1.date_index - t2.date_index <= 0 THEN t2.Merchant_State_isna ELSE NULL END) 
AS cardnum_state_isna_1

-- A self join is applied, in this case either left or inner join is fine 
FROM card_transactions t1 LEFT JOIN card_transactions t2

-- Matching condition 1: the same Cardnum
ON t1.Cardnum = t2.Cardnum

-- Matching condition 2: within the (largest) time interval
AND t1.date_index - t2.date_index BETWEEN 0 AND 29

-- Matching condition 3: If a Cardnum has multiple records on the same day,
--                       only match the records before it (presumably they are in chronological order)
AND t1.Recordnum >= t2.Recordnum

-- Now each record has matched all the corresponding records in the specified time window,
-- we can group by the record id to calculate the four aggregated metrics
GROUP BY t1.Recordnum
'''

In [46]:
start = time.time()
cardnum = pysql(sql_code)
end = time.time()
print ('time elipsed: {:.2f}'.format(end-start))

time elipsed: 45.47


In [47]:
cardnum.tail()

,Recordnum,cardnum_amount_avg_30,cardnum_amount_avg_15,cardnum_amount_avg_7,cardnum_amount_avg_3,cardnum_amount_avg_1,cardnum_amount_max_30,cardnum_amount_max_15,cardnum_amount_max_7,cardnum_amount_max_3,...,cardnum_state_30,cardnum_state_15,cardnum_state_7,cardnum_state_3,cardnum_state_1,cardnum_state_isna_30,cardnum_state_isna_15,cardnum_state_isna_7,cardnum_state_isna_3,cardnum_state_isna_1
96348,96704,84.790000,84.790000,84.790000,84.7900,84.790,84.79,84.79,84.79,84.79,...,1,1,1,1,1,0,0,0,0,0
96349,96705,628.382500,622.246154,628.050000,628.0500,401.875,1705.60,1705.60,1478.00,1478.00,...,4,4,3,3,2,0,0,0,0,0
96350,96706,442.190769,429.688333,429.688333,386.6375,363.560,1065.73,1065.73,1065.73,1065.73,...,7,3,3,2,1,0,0,0,0,0
96351,96707,865.614000,1261.490000,2202.030000,2202.0300,2202.030,2202.03,2202.03,2202.03,2202.03,...,3,3,1,1,1,0,0,0,0,0
96352,96708,322.175667,1760.616000,1076.251667,796.1040,554.640,6964.90,6964.90,3142.52,3142.52,...,9,6,3,2,1,0,0,0,0,0


In [48]:
cardnum['cardnum_amount_max_3'].tail(1)

96352    3142.52
Name: cardnum_amount_max_3, dtype: float64

In [49]:
cardnum['cardnum_amount_avg_3'].tail(1)

96352    796.104
Name: cardnum_amount_avg_3, dtype: float64

In [50]:
cardnum['cardnum_3'].tail(1)

96352    5
Name: cardnum_3, dtype: int64

In [51]:
last = data.query('date_index >= 362')

In [52]:
last.Cardnum.iloc[-1]

5142243247

In [53]:
last.query('Cardnum==5142243247')

,Recordnum,Cardnum,Date,Merchantnum,Merch_Description,Merchant_State,Merchant_Zip,Transtype,Amount,Fraud,date_index,new_merch,weekday,month,Amount_week_diff,Amount_month_diff,Merchantnum_isna,Merchant_State_isna,Merchant_Zip_isna,for_na
96205,96206,5142243247,2010-12-29,61563,A DAIGGER & CO INC,IL,60069,P,116.20,0,362,0,2,12,-322.979218,-284.176958,0,0,0,n_96206
96251,96252,5142243247,2010-12-29,9108347680006,STAPLES NATIONAL #471,NJ,7606,P,76.80,0,362,0,2,12,-362.379218,-323.576958,0,0,0,n_96252
96376,96377,5142243247,2010-12-29,9108347680000,STAPLES NATIONAL #471,NJ,7606,P,3142.52,0,362,0,2,12,2703.340782,2742.143042,0,0,0,n_96377
96397,96398,5142243247,2010-12-29,9108347680006,STAPLES NATIONAL #471,NJ,7606,P,90.36,0,362,0,2,12,-348.819218,-310.016958,0,0,0,n_96398
96707,96708,5142243247,2010-12-31,9108347680006,STAPLES NATIONAL #471,NJ,7606,P,554.64,0,364,0,4,12,186.534536,154.263042,0,0,0,n_96708


In [54]:
np.mean(last.query('Cardnum==5142243247')['Amount'])

796.104

In [55]:
sql_code_2 = '''
SELECT t1.Recordnum,

--- 0. basic statistics
-- 0.0. avg amount 30, 15, 7, 3, 1

AVG(t2.Amount) As merchantnum_amount_avg_30,
AVG(CASE WHEN t1.date_index - t2.date_index <= 14 THEN t2.Amount ELSE NULL END) As merchantnum_amount_avg_15,
AVG(CASE WHEN t1.date_index - t2.date_index <= 6 THEN t2.Amount ELSE NULL END) As merchantnum_amount_avg_7,
AVG(CASE WHEN t1.date_index - t2.date_index <= 2 THEN t2.Amount ELSE NULL END) As merchantnum_amount_avg_3,
AVG(CASE WHEN t1.date_index - t2.date_index <= 0 THEN t2.Amount ELSE NULL END) As merchantnum_amount_avg_1,

-- 0.1. max amount 30, 15, 7, 3, 1

MAX(t2.Amount) As merchantnum_amount_max_30,
MAX(CASE WHEN t1.date_index - t2.date_index <= 14 THEN t2.Amount ELSE NULL END) As merchantnum_amount_max_15,
MAX(CASE WHEN t1.date_index - t2.date_index <= 6 THEN t2.Amount ELSE NULL END) As merchantnum_amount_max_7,
MAX(CASE WHEN t1.date_index - t2.date_index <= 2 THEN t2.Amount ELSE NULL END) As merchantnum_amount_max_3,
MAX(CASE WHEN t1.date_index - t2.date_index <= 0 THEN t2.Amount ELSE NULL END) As merchantnum_amount_max_1,

-- 0.2. sum amount 30, 15, 7, 3, 1

SUM(t2.Amount) As merchantnum_amount_sum_30,
SUM(CASE WHEN t1.date_index - t2.date_index <= 14 THEN t2.Amount ELSE NULL END) As merchantnum_amount_sum_15,
SUM(CASE WHEN t1.date_index - t2.date_index <= 6 THEN t2.Amount ELSE NULL END) As merchantnum_amount_sum_7,
SUM(CASE WHEN t1.date_index - t2.date_index <= 2 THEN t2.Amount ELSE NULL END) As merchantnum_amount_sum_3,
SUM(CASE WHEN t1.date_index - t2.date_index <= 0 THEN t2.Amount ELSE NULL END) As merchantnum_amount_sum_1,

-- 0.3. avg amount week_diff 30, 15, 7, 3, 1

AVG(t2.Amount_week_diff) As merchantnum_amount_week_diff_avg_30,
AVG(CASE WHEN t1.date_index - t2.date_index <= 14 THEN t2.Amount_week_diff ELSE NULL END) 
As merchantnum_amount_week_diff_avg_15,
AVG(CASE WHEN t1.date_index - t2.date_index <= 6 THEN t2.Amount_week_diff ELSE NULL END) 
As merchantnum_amount_week_diff_avg_7,
AVG(CASE WHEN t1.date_index - t2.date_index <= 2 THEN t2.Amount_week_diff ELSE NULL END) 
As merchantnum_amount_week_diff_avg_3,
AVG(CASE WHEN t1.date_index - t2.date_index <= 0 THEN t2.Amount_week_diff ELSE NULL END) 
As merchantnum_amount_week_diff_avg_1,

-- 0.4. max amount week_diff 30, 15, 7, 3, 1

MAX(t2.Amount_week_diff) As merchantnum_amount_week_diff_max_30,
MAX(CASE WHEN t1.date_index - t2.date_index <= 14 THEN t2.Amount_week_diff ELSE NULL END) 
As merchantnum_amount_week_diff_max_15,
MAX(CASE WHEN t1.date_index - t2.date_index <= 6 THEN t2.Amount_week_diff ELSE NULL END) 
As merchantnum_amount_week_diff_max_7,
MAX(CASE WHEN t1.date_index - t2.date_index <= 2 THEN t2.Amount_week_diff ELSE NULL END) 
As merchantnum_amount_week_diff_max_3,
MAX(CASE WHEN t1.date_index - t2.date_index <= 0 THEN t2.Amount_week_diff ELSE NULL END) 
As merchantnum_amount_week_diff_max_1,

-- 0.5. sum amount week_diff 30, 15, 7, 3, 1

SUM(t2.Amount_week_diff) As merchantnum_amount_week_diff_sum_30,
SUM(CASE WHEN t1.date_index - t2.date_index <= 14 THEN t2.Amount_week_diff ELSE NULL END) 
As merchantnum_amount_week_diff_sum_15,
SUM(CASE WHEN t1.date_index - t2.date_index <= 6 THEN t2.Amount_week_diff ELSE NULL END) 
As merchantnum_amount_week_diff_sum_7,
SUM(CASE WHEN t1.date_index - t2.date_index <= 2 THEN t2.Amount_week_diff ELSE NULL END) 
As merchantnum_amount_week_diff_sum_3,
SUM(CASE WHEN t1.date_index - t2.date_index <= 0 THEN t2.Amount_week_diff ELSE NULL END) 
As merchantnum_amount_week_diff_sum_1,

-- 0.6. avg amount month_diff 30, 15, 7, 3, 1

AVG(t2.Amount_month_diff) As merchantnum_amount_month_diff_avg_30,
AVG(CASE WHEN t1.date_index - t2.date_index <= 14 THEN t2.Amount_month_diff ELSE NULL END) 
As merchantnum_amount_month_diff_avg_15,
AVG(CASE WHEN t1.date_index - t2.date_index <= 6 THEN t2.Amount_month_diff ELSE NULL END) 
As merchantnum_amount_month_diff_avg_7,
AVG(CASE WHEN t1.date_index - t2.date_index <= 2 THEN t2.Amount_month_diff ELSE NULL END) 
As merchantnum_amount_month_diff_avg_3,
AVG(CASE WHEN t1.date_index - t2.date_index <= 0 THEN t2.Amount_month_diff ELSE NULL END) 
As merchantnum_amount_month_diff_avg_1,

-- 0.7. max amount month_diff 30, 15, 7, 3, 1

MAX(t2.Amount_month_diff) As merchantnum_amount_month_diff_max_30,
MAX(CASE WHEN t1.date_index - t2.date_index <= 14 THEN t2.Amount_month_diff ELSE NULL END) 
As merchantnum_amount_month_diff_max_15,
MAX(CASE WHEN t1.date_index - t2.date_index <= 6 THEN t2.Amount_month_diff ELSE NULL END) 
As merchantnum_amount_month_diff_max_7,
MAX(CASE WHEN t1.date_index - t2.date_index <= 2 THEN t2.Amount_month_diff ELSE NULL END) 
As merchantnum_amount_month_diff_max_3,
MAX(CASE WHEN t1.date_index - t2.date_index <= 0 THEN t2.Amount_month_diff ELSE NULL END) 
As merchantnum_amount_month_diff_max_1,

-- 0.8. sum amount month_diff 30, 15, 7, 3, 1

SUM(t2.Amount_month_diff) As merchantnum_amount_month_diff_sum_30,
SUM(CASE WHEN t1.date_index - t2.date_index <= 14 THEN t2.Amount_month_diff ELSE NULL END) 
As merchantnum_amount_month_diff_sum_15,
SUM(CASE WHEN t1.date_index - t2.date_index <= 6 THEN t2.Amount_month_diff ELSE NULL END) 
As merchantnum_amount_month_diff_sum_7,
SUM(CASE WHEN t1.date_index - t2.date_index <= 2 THEN t2.Amount_month_diff ELSE NULL END) 
As merchantnum_amount_month_diff_sum_3,
SUM(CASE WHEN t1.date_index - t2.date_index <= 0 THEN t2.Amount_month_diff ELSE NULL END) 
As merchantnum_amount_month_diff_sum_1,

--- 1. burst
-- 1.0. counting the same card number in the past 30,15,7,3,1 days

COUNT(*) AS merchantnum_30,
SUM(CASE WHEN t1.date_index - t2.date_index <= 14 THEN 1 ELSE NULL END) AS merchantnum_15,
SUM(CASE WHEN t1.date_index - t2.date_index <= 6 THEN 1 ELSE NULL END) AS merchantnum_7,
SUM(CASE WHEN t1.date_index - t2.date_index <= 2 THEN 1 ELSE NULL END) AS merchantnum_3,
SUM(CASE WHEN t1.date_index - t2.date_index <= 0 THEN 1 ELSE NULL END) AS merchantnum_1,

--- 2. Cardnum
-- 2.0. how many different Cardnum for a given Merchantnum in 30,15,7,3,1 days 

COUNT(DISTINCT t2.Cardnum) AS merchantnum_cardnum_30,
COUNT(DISTINCT CASE WHEN t1.date_index - t2.date_index <= 14 THEN t2.Cardnum ELSE NULL END) 
AS merchantnum_cardnum_15,
COUNT(DISTINCT CASE WHEN t1.date_index - t2.date_index <= 6 THEN t2.Cardnum ELSE NULL END) 
AS merchantnum_cardnum_7,
COUNT(DISTINCT CASE WHEN t1.date_index - t2.date_index <= 2 THEN t2.Cardnum ELSE NULL END) 
AS merchantnum_cardnum_3,
COUNT(DISTINCT CASE WHEN t1.date_index - t2.date_index <= 0 THEN t2.Cardnum ELSE NULL END) 
AS merchantnum_cardnum_1,


--- 3. Zip
-- 3.0. how many different zip for a given Merchantnum in 30,15,7,3,1 days 

COUNT(DISTINCT t2.Merchant_Zip) AS merchantnum_zip_30,
COUNT(DISTINCT CASE WHEN t1.date_index - t2.date_index <= 14 THEN t2.Merchant_Zip ELSE NULL END) 
AS merchantnum_zip_15,
COUNT(DISTINCT CASE WHEN t1.date_index - t2.date_index <= 6 THEN t2.Merchant_Zip ELSE NULL END) 
AS merchantnum_zip_7,
COUNT(DISTINCT CASE WHEN t1.date_index - t2.date_index <= 2 THEN t2.Merchant_Zip ELSE NULL END) 
AS merchantnum_zip_3,
COUNT(DISTINCT CASE WHEN t1.date_index - t2.date_index <= 0 THEN t2.Merchant_Zip ELSE NULL END) 
AS merchantnum_zip_1,

-- 3.1. how many originally na zip for a given Merchantnum in 30,15,7,3,1 days 

SUM(t2.Merchant_Zip_isna) AS merchantnum_zip_isna_30,
SUM(DISTINCT CASE WHEN t1.date_index - t2.date_index <= 14 THEN t2.Merchant_Zip_isna ELSE NULL END) 
AS merchantnum_zip_isna_15,
SUM(DISTINCT CASE WHEN t1.date_index - t2.date_index <= 6 THEN t2.Merchant_Zip_isna ELSE NULL END) 
AS merchantnum_zip_isna_7,
SUM(DISTINCT CASE WHEN t1.date_index - t2.date_index <= 2 THEN t2.Merchant_Zip_isna ELSE NULL END) 
AS merchantnum_zip_isna_3,
SUM(DISTINCT CASE WHEN t1.date_index - t2.date_index <= 0 THEN t2.Merchant_Zip_isna ELSE NULL END) 
AS merchantnum_zip_isna_1,

--- 4. states
-- 4.0. how many different states for a given Merchantnum in 30,15,7,3,1 days 

COUNT(DISTINCT t2.Merchant_State) AS merchantnum_state_30,
COUNT(DISTINCT CASE WHEN t1.date_index - t2.date_index <= 14 THEN t2.Merchant_State ELSE NULL END) 
AS merchantnum_state_15,
COUNT(DISTINCT CASE WHEN t1.date_index - t2.date_index <= 6 THEN t2.Merchant_State ELSE NULL END) 
AS merchantnum_state_7,
COUNT(DISTINCT CASE WHEN t1.date_index - t2.date_index <= 2 THEN t2.Merchant_State ELSE NULL END) 
AS merchantnum_state_3,
COUNT(DISTINCT CASE WHEN t1.date_index - t2.date_index <= 0 THEN t2.Merchant_State ELSE NULL END) 
AS merchantnum_state_1,

-- 5.1. how many originally na states for a given Merchantnum in 30,15,7,3,1 days 

SUM(t2.Merchant_State_isna) AS merchantnum_state_isna_30,
SUM(CASE WHEN t1.date_index - t2.date_index <= 14 THEN t2.Merchant_State_isna ELSE NULL END) 
AS merchantnum_state_isna_15,
SUM(CASE WHEN t1.date_index - t2.date_index <= 6 THEN t2.Merchant_State_isna ELSE NULL END) 
AS merchantnum_state_isna_7,
SUM(CASE WHEN t1.date_index - t2.date_index <= 2 THEN t2.Merchant_State_isna ELSE NULL END) 
AS merchantnum_state_isna_3,
SUM(CASE WHEN t1.date_index - t2.date_index <= 0 THEN t2.Merchant_State_isna ELSE NULL END) 
AS merchantnum_state_isna_1



-- A self join is applied, in this case either left or inner join is fine 
FROM card_transactions t1 LEFT JOIN card_transactions t2

-- Matching condition 1: the same Merchantnum
ON t1.Merchantnum = t2.Merchantnum

-- Matching condition 2: within the (largest) time interval
AND t1.date_index - t2.date_index BETWEEN 0 AND 29

-- Matching condition 3: If a Merchantnum has multiple records on the same day,
--                       only match the records before it (presumably they are in chronological order)
AND t1.Recordnum >= t2.Recordnum

-- Now each record has matched all the corresponding records in the specified time window,
-- we can group by the record id to calculate the four aggregated metrics
GROUP BY t1.Recordnum
'''

In [56]:
start = time.time()
merchantnum = pysql(sql_code_2)
end = time.time()
print ('time elipsed: {:.2f}'.format(end-start))

time elipsed: 166.86


In [57]:
merchantnum.tail()

,Recordnum,merchantnum_amount_avg_30,merchantnum_amount_avg_15,merchantnum_amount_avg_7,merchantnum_amount_avg_3,merchantnum_amount_avg_1,merchantnum_amount_max_30,merchantnum_amount_max_15,merchantnum_amount_max_7,merchantnum_amount_max_3,...,merchantnum_state_30,merchantnum_state_15,merchantnum_state_7,merchantnum_state_3,merchantnum_state_1,merchantnum_state_isna_30,merchantnum_state_isna_15,merchantnum_state_isna_7,merchantnum_state_isna_3,merchantnum_state_isna_1
96348,96704,84.790000,84.790000,84.7900,84.7900,84.7900,84.79,84.79,84.79,84.79,...,1,1,1,1,1,0,0,0,0,0
96349,96705,391.494286,324.080000,118.7500,118.7500,118.7500,1050.12,763.53,118.75,118.75,...,1,1,1,1,1,0,0,0,0,0
96350,96706,1191.567692,954.861429,1073.7050,1445.2225,1445.2225,2494.40,2487.98,2487.98,2487.98,...,1,1,1,1,1,0,0,0,0,0
96351,96707,729.972128,716.041538,1430.3075,1430.3075,2202.0300,5013.39,2349.95,2312.55,2312.55,...,1,1,1,1,1,0,0,0,0,0
96352,96708,106.438462,146.673333,240.6000,240.6000,554.6400,554.64,554.64,554.64,554.64,...,1,1,1,1,1,0,0,0,0,0


## Data cleaning

before correcting missing value encodings

In [58]:
cardnum[['cardnum_merchantnum_30','cardnum_zip_30','cardnum_state_30']].describe()

,cardnum_merchantnum_30,cardnum_zip_30,cardnum_state_30
count,96353.000000,96353.000000,96353.000000
mean,9.017010,8.670555,5.537056
std,8.451985,8.315366,4.076208
min,1.000000,1.000000,1.000000
25%,4.000000,3.000000,3.000000
50%,7.000000,6.000000,5.000000
75%,12.000000,11.000000,7.000000
max,86.000000,114.000000,31.000000


In [59]:
lags = ['1','3','7','15','30']
card_missing_reset = ['merchantnum','zip','state']
merchant_missing_reset = ['zip','state']
for l in lags:
    for c in card_missing_reset:
        cardnum['cardnum_{}_{}'.format(c,l)] = (cardnum['cardnum_{}_{}'.format(c,l)]-
                                               cardnum['cardnum_{}_isna_{}'.format(c,l)])
    for m in merchant_missing_reset:
        merchantnum['merchantnum_{}_{}'.format(m,l)] = (merchantnum['merchantnum_{}_{}'.format(m,l)]-
                                               merchantnum['merchantnum_{}_isna_{}'.format(m,l)])

after correcting missing value encodings

In [60]:
cardnum[['cardnum_merchantnum_30','cardnum_zip_30','cardnum_state_30']].describe()

,cardnum_merchantnum_30,cardnum_zip_30,cardnum_state_30
count,96353.000000,96353.000000,96353.000000
mean,8.538561,7.934346,5.399707
std,7.914273,7.331482,3.910024
min,0.000000,0.000000,0.000000
25%,3.000000,3.000000,2.000000
50%,6.000000,6.000000,5.000000
75%,11.000000,10.000000,7.000000
max,80.000000,73.000000,30.000000


## Data Aggregating

In [61]:
data.head()

,Recordnum,Cardnum,Date,Merchantnum,Merch_Description,Merchant_State,Merchant_Zip,Transtype,Amount,Fraud,date_index,new_merch,weekday,month,Amount_week_diff,Amount_month_diff,Merchantnum_isna,Merchant_State_isna,Merchant_Zip_isna,for_na
0,1,5142190439,2010-01-01,5509006296254,FEDEX SHP 12/23/09 AB#,TN,38118,P,3.62,0,0,1,4,1,-364.485464,-349.932108,0,0,0,n_1
1,2,5142183973,2010-01-01,61003026333,SERVICE MERCHANDISE #81,MA,1803,P,31.42,0,0,1,4,1,-336.685464,-322.132108,0,0,0,n_2
2,3,5142131721,2010-01-01,4503082993600,OFFICE DEPOT #191,MD,20706,P,178.49,0,0,1,4,1,-189.615464,-175.062108,0,0,0,n_3
3,4,5142148452,2010-01-01,5509006296254,FEDEX SHP 12/28/09 AB#,TN,38118,P,3.62,0,0,0,4,1,-364.485464,-349.932108,0,0,0,n_4
4,5,5142190439,2010-01-01,5509006296254,FEDEX SHP 12/23/09 AB#,TN,38118,P,3.62,0,0,0,4,1,-364.485464,-349.932108,0,0,0,n_5


In [65]:
output = pd.concat([cardnum.set_index('Recordnum'),
                    merchantnum.set_index('Recordnum'),
                    data[['Recordnum','Fraud','date_index']].set_index('Recordnum')],axis=1)

In [66]:
output.shape

(96353, 162)

In [70]:
dates[303]

'2010-10-31'

In [74]:
train = output.query('date_index <= 303')
train = train.iloc[:,:-1]

In [75]:
out_of_date = output.query('date_index > 303')
out_of_date = out_of_date.iloc[:,:-1]

In [76]:
train.shape

(83767, 161)

In [77]:
out_of_date.shape

(12586, 161)

In [79]:
train.tail()

,cardnum_amount_avg_30,cardnum_amount_avg_15,cardnum_amount_avg_7,cardnum_amount_avg_3,cardnum_amount_avg_1,cardnum_amount_max_30,cardnum_amount_max_15,cardnum_amount_max_7,cardnum_amount_max_3,cardnum_amount_max_1,...,merchantnum_state_15,merchantnum_state_7,merchantnum_state_3,merchantnum_state_1,merchantnum_state_isna_30,merchantnum_state_isna_15,merchantnum_state_isna_7,merchantnum_state_isna_3,merchantnum_state_isna_1,Fraud
Recordnum,,,,,,,,,,,,,,,,,,,,,
84091,344.3800,299.770000,299.770,299.77,299.77,929.00,299.77,299.77,299.77,299.77,...,1,1,1,1,0,0,0,0,0,0
84092,394.6600,394.660000,394.660,394.66,394.66,609.34,609.34,609.34,609.34,609.34,...,1,1,1,1,0,0,0,0,0,0
84093,221.5875,218.033333,127.475,235.00,235.00,399.15,399.15,235.00,235.00,235.00,...,1,1,1,1,0,0,0,0,0,0
84094,600.0000,600.000000,600.000,600.00,600.00,600.00,600.00,600.00,600.00,600.00,...,1,1,1,1,0,0,0,0,0,0
84095,197.6600,30.000000,30.000,30.00,30.00,365.32,30.00,30.00,30.00,30.00,...,1,1,1,1,0,0,0,0,0,0


In [80]:
train.to_csv(path+'train.csv')
out_of_date.to_csv(path+'out_of_date.csv')